In [1]:
from ngsolve import *
from ngsolve import Draw, curl, grad, krylovspace
from netgen.occ import *
from netgen.gui import *
from coil_geometry import *
import numpy as np 

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2405
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 14 thread(s)


2024-12-30 23:46:12.822 Python[35116:172504] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-30 23:46:12.822 Python[35116:172504] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [2]:
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilradius = 0.01  # Radius of the coil

# Geometrie laden
geo,_ = create_homo_geometry(nwindings, wireradius, coilradius)

In [3]:
Draw(geo)

In [4]:
mesh = Mesh(
    geo.GenerateMesh()
)
mesh.Curve(1) 

 Face 1 / 10 (parameter space projection)
 Face 2 / 10 (parameter space projection)
 Face 3 / 10 (parameter space projection)
 Face 4 / 10 (parameter space projection)
 Face 5 / 10 (parameter space projection)
 Face 6 / 10 (parameter space projection)
 Face 7 / 10 (parameter space projection)
 Face 8 / 10 (parameter space projection)
 Face 9 / 10 (parameter space projection)
 Face 10 / 10 (parameter space projection)
 Delaunay meshing
 Remove Illegal Elements
 Delaunay meshing
 start tetmeshing
 Success !
 Volume Optimization
 Curve elements, order = 1


In [5]:
print("Materialien im Mesh:", mesh.GetMaterials())
print("Randbedingungen im Mesh:", mesh.GetBoundaries())

Materialien im Mesh: ('coil', 'air')
Randbedingungen im Mesh: ('default', 'default', 'default', 'default', 'outer', 'outer', 'outer', 'outer', 'outer', 'outer')


In [6]:
# Konstanten
mu = 4 * pi * 1e-7      # Magnetische Feldkonstante (S.2)
sigma_coil = 5.87e7     # Leitfähigkeit Kupfer (S.11)
sigma_air = 0           # Leitfähigkeit Luft
k = 0.1                 # Regularisierungfaktor
i_c = 500               # Strom

# Geometrische Parameter
A_c = pi * wireradius**2  
r = sqrt(x**2 + y**2 + z**2)

# Stromdichte j_c gemäß Aufgabenstellung
j_c = CoefficientFunction((
    (nwindings * i_c / A_c) * y / r,
    -(nwindings * i_c / A_c) * x / r,
    0
))

In [7]:
# H(curl)-Raum definieren
order = 2
V = HCurl(mesh, order=order-1, nograds=True, dirichlet="outer")
u, v = V.TnT()
gfA = GridFunction(V)


# Bilinearform und Linearform
a = BilinearForm(V, symmetric=True)
a += (1/mu) * InnerProduct(curl(u), curl(v)) * dx
a += k * InnerProduct(u, v) * dx

# Preconditioner
pre = Preconditioner(a, "multigrid")

# Linearform: Stromdichte
f = LinearForm(V)
f += InnerProduct(j_c, v) * dx("coil")

# Assemble
with TaskManager():
    a.Assemble()
    f.Assemble()

hcurl smoothingblocks, SmoothingType = 2


In [8]:
# Grösse von Gleichungsystems Elemente überprüfen
print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")
print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")

Matrix size: 47895 x 47895
Vector size: 47895
Präkonditionierer-Matrixgröße: 47895 x 47895


In [9]:
# Lösen des Gleichungssystems mit CG-Verfahren und Präkonditionierer (krylovspace.CGSolver)
inv = krylovspace.CGSolver(a.mat, pre.mat, tol=1e-12, printrates = True)

with TaskManager():
    gfA.vec.data = inv * f.vec

# Visualisierung
Draw(curl(gfA), mesh, "MagneticField")
Draw(gfA, mesh, "VectorPotential", draw_surf=True, draw_vol=True)

CG iteration 1, residual = 1072.7075249396498     
CG iteration 2, residual = 4.04042686083724     
CG iteration 3, residual = 0.023169325996392782     
CG iteration 4, residual = 0.00017029861977906896     
CG iteration 5, residual = 1.5286487254315575e-06     
CG iteration 6, residual = 1.50984883099077e-08     
CG iteration 7, residual = 1.6580150577658249e-10     


In [10]:
# Calculate the residual manually
residual_vec = f.vec - a.mat * gfA.vec
residual_norm = residual_vec.Norm()
print(f"Residual norm: {residual_norm}")

Residual norm: 1345.1772467439348


In [11]:
B = curl(gfA) 

In [12]:
# Feldlinien to be done 